# 한국어를 활용한 Byte-Pair Encoding

2015년 **Rico Sennrich**가 작성한 논문 [**Neural Machine Translation of Rare Words with Subword Units**](https://arxiv.org/abs/1508.07909)에서는<br/>**OOV (Out-Of-Vocabulary)** 문제를 해결하기 위해 **정보 이론**에서의 **Byte-Pair Encoding** 기법을 토크나이즈에 접목시킨 **BPE Embedding**이 고안되었습니다.

해당 알고리즘 덕분에 **자연어 처리**의 많은 모델들은 **OOV** 문제를 해결할 수 있게 되었고, 뿐만 아니라 **성능 개선**까지 얻게 되었습니다.

그리고 **BPE**를 활용한 토크나이즈 기법은 **Transformer** 기반의 여러 최신 모델들에서 아직까지 차용되고 있습니다.

본 노트북에서는 이러한 **BPE** 임베딩을 **한국어**에 적용해보며, 그 특성과 구현 방법에 대해 알아보도록 하겠습니다.

## 정규표현식을 통한 전처리

먼저 **정규표현식**을 활용해 한국어 문장을 전처리 해줄 수 있는 `preprocess` 함수를 구현해보도록 합시다.

한국어 전처리를 위한 정규표현식을 어떻게 구성할지는 **프로젝트의 목적**에 따라 다를 수 있습니다.

영어, 숫자 등이 의미를 큰 지니지 않는 코퍼스에 대해서는 해당 캐릭터들을 모두 제거할 수도 있겠지만, <br/>
영어, 숫자 등이 의미를 지니는 경우 해당 캐릭터들을 제거하지 않을 수도 있습니다.

본 예제에서는 **한국어**의 **Byte-Pair**가 어떻게 형성되는지 보기 위해 **불필요한 특수문자와 숫자, 영어를 모두 제거**하도록 하겠습니다.

In [1]:
import re
from collections import defaultdict
from typing import Dict, List, Tuple
from tqdm import tqdm


SPECIALS = "".join([".", ",", ";", ":", "!", "?", '"', "'", " "])


def preprocess(text: str, only_kor: bool=True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|{SPECIALS}|a-zA-Z|]+", "", text)
    
    # 연속 공백 제거
    text = re.sub(" +", " ", text)
    
    # 좌우 불필요한 공백 제거
    return text.strip()

In [2]:
sent = "ㅋㅋㅋ 안녕하세요 ! \"저는\" 10년차 original 두산 팬입니다."

In [3]:
preprocess(sent)

'안녕하세요 저는 년차 두산 팬입니다'

In [4]:
preprocess(sent, only_kor=False)

'ㅋㅋㅋ 안녕하세요 ! "저는" 10년차 original 두산 팬입니다.'

## Byte-Pair Encoding 구현

이제 전처리를 거친 문장들로 구성된 코퍼스를 활용해 **Byte-Pair**를 구성할 수 있는 **Encoding** 로직을 구현해보도록 합시다.

**Byte-Pair Encoding**은 다음과 같은 **3개의 핵심 스텝**으로 구현될 수 있습니다.

1. `get_vocab`
2. `get_stats`
3. `merge_vocab`

_cf. 해당 코드는 **Lei Mao**의 [포스트](https://leimao.github.io/blog/Byte-Pair-Encoding/)에서 차용 및 수정하였습니다._

3개의 스텝 중 첫 번째 스텝인 `get_vocab` 함수에 대해 먼저 살펴봅시다.

해당 함수가 수행하는 로직은 간단합니다.

1. 코퍼스 파일을 **라인 단위**로 읽어옵니다.
2. 각 라인을 **공백 단위로 스플릿**해 토큰 리스트를 구성합니다.
3. 토큰 리스트를 순회하며, 토큰을 **캐릭터 단위로 자른 후** 사전에 postfix `</w>`와 함께 등록 (혹은 +1) 합니다.

_cf. 논문에서는 공백과 함께 `</w>`를 끝에 더해주었지만, 0.2 버전 이후로는 공백 없이 `</w>`를 붙여줍니다._

_Since 0.2, **end-of-word tokens are initially concatenated** with the word-final character:_

```
u n d</w>
f u n d</w>
```

_The new representation ensures that when BPE codes are learned from the above examples and then applied to new text, it is clear that a subword unit `und` is unambiguously word-final, and `un` is unambiguously word-internal, **preventing the production of up to two different subword units from each BPE merge operation.**_

In [5]:
def get_vocab(f_name: str) -> Dict[str, int]:
    """코퍼스 파일을 읽어와 단어 사전 구축"""
    vocab = defaultdict(int)
    with open(f_name, "r", encoding="utf-8") as corpus:
        for line in corpus:
            tokens = preprocess(line).strip().split()
            for token in tokens:
                vocab[" ".join(list(token)) + "</w>"] += 1
    return dict(vocab)

전체 코퍼스를 활용해 사전을 구축하기 전에, 간단한 **더미 데이터**로 실험을 하기 위한 `pseudo_get_vocab` 함수를 구현해주겠습니다.

In [6]:
def pseudo_get_vocab(corpus: List[str]) -> Dict[str, int]:
    """더미 데이터를 읽어와 단어 사전 구축"""
    vocab = defaultdict(int)
    for line in corpus:
        tokens = preprocess(line).strip().split(" ")
        for token in tokens:
            vocab[" ".join(list(token)) + " </w>"] += 1
    return dict(vocab)

In [7]:
corpus = [
    "기술을 사랑해",
    "축구를 사랑해",
    "한국을 사랑해",
    "야구는 사랑하지 않아",
    "삽질은 사랑하지 않아",
    "내가 사랑하는 너",
    "네가 사랑하는 나"
]

In [8]:
vocab = pseudo_get_vocab(corpus)
vocab

{'기 술 을 </w>': 1,
 '사 랑 해 </w>': 3,
 '축 구 를 </w>': 1,
 '한 국 을 </w>': 1,
 '야 구 는 </w>': 1,
 '사 랑 하 지 </w>': 2,
 '않 아 </w>': 2,
 '삽 질 은 </w>': 1,
 '내 가 </w>': 1,
 '사 랑 하 는 </w>': 2,
 '너 </w>': 1,
 '네 가 </w>': 1,
 '나 </w>': 1}

이제 구축된 사전을 순회하며 사전 내 등록된 **캐릭터 토큰**과 **등장 횟수**를 반환해주는 함수 `get_tokens`를 구현해줍니다.

이후, **Byte-Pair Encoding** 로직을 거치기 이전의 사전을 확인합니다.

In [9]:
def get_tokens(vocab: Dict[str, int]):
    """사전 내 등록된 토큰을 확인"""
    result = defaultdict(int)
    for word, freq in vocab.items():
        tokens = word.split()
        for token in tokens:
            result[token] += freq
    return dict(result)

In [10]:
tokens = get_tokens(vocab)
tokens

{'기': 1,
 '술': 1,
 '을': 2,
 '</w>': 18,
 '사': 7,
 '랑': 7,
 '해': 3,
 '축': 1,
 '구': 2,
 '를': 1,
 '한': 1,
 '국': 1,
 '야': 1,
 '는': 3,
 '하': 4,
 '지': 2,
 '않': 2,
 '아': 2,
 '삽': 1,
 '질': 1,
 '은': 1,
 '내': 1,
 '가': 2,
 '너': 1,
 '네': 1,
 '나': 1}

마지막 스텝인 `merge_vocab`은 **가장 자주 등장했던 바이그램 페어**를 엮어주는 함수여야하므로,

**가장 자주 등장한 페어**를 구할 수 있는 로직 `get_stats` 함수가 필요합니다.

In [11]:
def get_stats(vocab: Dict[str, int]):
    """사전을 활용한 바이그램 페어 구축"""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return dict(pairs)

In [12]:
pairs = get_stats(vocab)
pairs

{('기', '술'): 1,
 ('술', '을'): 1,
 ('을', '</w>'): 2,
 ('사', '랑'): 7,
 ('랑', '해'): 3,
 ('해', '</w>'): 3,
 ('축', '구'): 1,
 ('구', '를'): 1,
 ('를', '</w>'): 1,
 ('한', '국'): 1,
 ('국', '을'): 1,
 ('야', '구'): 1,
 ('구', '는'): 1,
 ('는', '</w>'): 3,
 ('랑', '하'): 4,
 ('하', '지'): 2,
 ('지', '</w>'): 2,
 ('않', '아'): 2,
 ('아', '</w>'): 2,
 ('삽', '질'): 1,
 ('질', '은'): 1,
 ('은', '</w>'): 1,
 ('내', '가'): 1,
 ('가', '</w>'): 2,
 ('하', '는'): 2,
 ('너', '</w>'): 1,
 ('네', '가'): 1,
 ('나', '</w>'): 1}

이제 3개 스텝 중 **마지막 함수**인 `merge_vocab` 함수를 구현해줍니다.

`merge_vocab`의 로직 역시 간단합니다.

1. `get_stats` 함수를 통해 얻어진 바이그램 중 **가장 자주 등장한 페어**와 **기존 사전**을 인자로 받습니다.
2. 기존 사전에서 **캐릭터 단위**로 떨어져 있던 **바이그램을 하나의 페어**로 엮어줍니다.
3. **새로이 구축한 사전**을 반환합니다.

In [13]:
def merge_vocab(pair: Tuple[str, str], vocab: Dict[str, int]):
    """가장 자주 등장한 바이그램 페어를 엮어줌"""
    result = defaultdict(int)
    for word in vocab:
        paired = word.replace(" ".join(pair), "".join(pair))
        result[paired] = vocab[word]
    return dict(result)

지금까지 구현한 함수들을 활용해 자주 등장한 **Byte-Pair**가 합쳐져, 새로운 사전이 구축되는 과정을 살펴보도록 합시다.

_cf. `num_merges` 인자에 따라 **사전의 전체 크기** 등을 조정할 수 있습니다._

In [14]:
num_merges = 5

for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    tokens = get_tokens(vocab)
    print(f"Iter: {i+1}\n"
          f"Best pair: {best}\n"
          f"Tokens: {tokens}\n"
          f"Number of tokens: {len(tokens)}\n")

Iter: 1
Best pair: ('사', '랑')
Tokens: {'기': 1, '술': 1, '을': 2, '</w>': 18, '사랑': 7, '해': 3, '축': 1, '구': 2, '를': 1, '한': 1, '국': 1, '야': 1, '는': 3, '하': 4, '지': 2, '않': 2, '아': 2, '삽': 1, '질': 1, '은': 1, '내': 1, '가': 2, '너': 1, '네': 1, '나': 1}
Number of tokens: 25

Iter: 2
Best pair: ('사랑', '하')
Tokens: {'기': 1, '술': 1, '을': 2, '</w>': 18, '사랑': 3, '해': 3, '축': 1, '구': 2, '를': 1, '한': 1, '국': 1, '야': 1, '는': 3, '사랑하': 4, '지': 2, '않': 2, '아': 2, '삽': 1, '질': 1, '은': 1, '내': 1, '가': 2, '너': 1, '네': 1, '나': 1}
Number of tokens: 25

Iter: 3
Best pair: ('사랑', '해')
Tokens: {'기': 1, '술': 1, '을': 2, '</w>': 18, '사랑해': 3, '축': 1, '구': 2, '를': 1, '한': 1, '국': 1, '야': 1, '는': 3, '사랑하': 4, '지': 2, '않': 2, '아': 2, '삽': 1, '질': 1, '은': 1, '내': 1, '가': 2, '너': 1, '네': 1, '나': 1}
Number of tokens: 24

Iter: 4
Best pair: ('사랑해', '</w>')
Tokens: {'기': 1, '술': 1, '을': 2, '</w>': 15, '사랑해</w>': 3, '축': 1, '구': 2, '를': 1, '한': 1, '국': 1, '야': 1, '는': 3, '사랑하': 4, '지': 2, '않': 2, '아': 2, '삽': 1, '질': 1, '은'

이제 제대로 된 훈련 파일을 읽어와 `vocab`을 구축해보도록 합시다.

훈련 파일로는 [**NSMC**](https://github.com/e9t/nsmc)의 훈련 데이터셋을 이용하도록 하겠습니다.

In [15]:
vocab = get_vocab("data/ratings_train.txt")
vocab

{'아</w>': 1701,
 '더 빙</w>': 128,
 '진 짜</w>': 6280,
 '짜 증 나 네 요</w>': 17,
 '목 소 리</w>': 108,
 '흠 포 스 터 보 고</w>': 1,
 '초 딩 영 화 줄 오 버 연 기 조 차</w>': 1,
 '가 볍 지</w>': 17,
 '않 구 나</w>': 2,
 '너 무 재 밓 었 다 그 래 서 보 는 것 을 추 천 한 다</w>': 1,
 '교 도 소</w>': 4,
 '이 야 기 구 먼</w>': 1,
 '솔 직 히</w>': 946,
 '재 미 는</w>': 294,
 '없 다 평 점</w>': 1,
 '조 정</w>': 11,
 '사 이 몬 페 그 의</w>': 1,
 '익 살 스 런</w>': 2,
 '연 기 가</w>': 747,
 '돋 보 였 던</w>': 23,
 '영 화 스 파 이 더 맨 에 서</w>': 1,
 '늙 어 보 이 기 만</w>': 1,
 '했 던</w>': 138,
 '커 스 틴</w>': 4,
 '던 스 트 가</w>': 1,
 '너 무 나 도</w>': 193,
 '이 뻐 보 였 다</w>': 1,
 '막</w>': 192,
 '걸 음 마</w>': 1,
 '뗀</w>': 2,
 '세 부 터</w>': 1,
 '초 등 학 교</w>': 53,
 '학 년 생 인</w>': 1,
 '살 용 영 화 별 반 개 도</w>': 1,
 '아 까 움</w>': 263,
 '원 작 의</w>': 110,
 '긴 장 감 을</w>': 56,
 '제 대 로</w>': 653,
 '살 려 내 지 못 했 다</w>': 1,
 '별</w>': 737,
 '반 개 도</w>': 97,
 '아 깝 다</w>': 1196,
 '욕 나 온 다</w>': 35,
 '이 응 경</w>': 3,
 '길 용 우</w>': 1,
 '연 기 생 활 이 몇 년 인 지 정 말</w>': 1,
 '발 로 해 도</w>': 1,
 '그 것 보 단</w>': 4,
 '낫 겟 다</w>': 2,
 '납 치 감 

구축한 사전에 `merge_vocab` 로직을 적용합니다. 간단한 실험을 위해 `num_merges`는 **1000**으로 설정하겠습니다.

In [16]:
num_merges = 1000

for i in tqdm(range(num_merges)):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:15<00:00,  1.09s/it]


In [17]:
vocab

{'아</w>': 1701,
 '더빙</w>': 128,
 '진짜</w>': 6280,
 '짜증 나 네요</w>': 17,
 '목 소리</w>': 108,
 '흠 포스터 보고</w>': 1,
 '초딩 영화 줄 오 버 연기 조 차</w>': 1,
 '가 볍 지</w>': 17,
 '않 구나</w>': 2,
 '너무 재 밓 었 다 그래 서 보는 것 을 추천 한다</w>': 1,
 '교 도 소</w>': 4,
 '이야기 구 먼</w>': 1,
 '솔직히</w>': 946,
 '재미는</w>': 294,
 '없다 평점</w>': 1,
 '조 정</w>': 11,
 '사이 몬 페 그 의</w>': 1,
 '익 살 스 런</w>': 2,
 '연기가</w>': 747,
 '돋 보 였 던</w>': 23,
 '영화 스 파이 더 맨 에서</w>': 1,
 '늙 어 보이 기 만</w>': 1,
 '했던</w>': 138,
 '커 스 틴</w>': 4,
 '던 스트 가</w>': 1,
 '너무 나도</w>': 193,
 '이 뻐 보 였다</w>': 1,
 '막</w>': 192,
 '걸 음 마</w>': 1,
 '뗀</w>': 2,
 '세 부터</w>': 1,
 '초 등 학교</w>': 53,
 '학 년 생 인</w>': 1,
 '살 용 영화 별 반 개도</w>': 1,
 '아까움</w>': 263,
 '원작 의</w>': 110,
 '긴장감 을</w>': 56,
 '제대로</w>': 653,
 '살 려 내 지 못 했다</w>': 1,
 '별</w>': 737,
 '반 개도</w>': 97,
 '아깝다</w>': 1196,
 '욕 나온다</w>': 35,
 '이 응 경</w>': 3,
 '길 용 우</w>': 1,
 '연기 생 활 이 몇 년 인 지 정말</w>': 1,
 '발 로 해도</w>': 1,
 '그 것 보단</w>': 4,
 '낫 겟 다</w>': 2,
 '납 치 감 금 만 반 복 반 복 이 드라마 는</w>': 1,
 '가족 도 없다</w>': 1,
 '연기 못 하는 

**BPE**를 활용한 토크나이즈는 **최장 길이 토큰의 매칭**을 우선적으로 적용하기 때문에,

사전을 **단어 길이 기준**의 **내림차순**으로 정렬해줘야 합니다. 이를 위해 `get_token_len` 함수를 구현해줍니다.

In [18]:
def get_token_len(token: str):
    """토큰 길이 계산: </w> 는 하나의 토큰 취급"""
    if token.endswith("</w>"):
        # 구성 캐릭터 + </w>
        return len(token[:-4]) + 1
    return len(token)

이제 구축된 사전을 활용해 새로운 입력 값을 토크나이즈 할 수 있는 `tokenize` 함수를 구현하도록 합니다.

**BPE**를 활용한 문장의 인코딩은 **재귀적으로** 구성되기 때문에, 직접 결과 값을 찍어보며 이해하는 것이 가장 좋습니다.

`tokenize`의 기본적인 로직은 다음과 같습니다.

1. 훈련을 통해 구축된 **단어 리스트**를 순회하며, 입력 문장 내 해당 단어가 존재하는지 파악합니다.
2. 순회하던 단어가 입력 문장 내에 **존재하지 않는다면**, 계속 순회를 진행합니다.
    - 만약 단어 리스트를 끝까지 순회했음에도 매치되는 토큰이 없었다면, `</u>`의 **OOV** 토큰을 반환합니다.
2. 순회하던 단어가 입력 문장 내에 **존재한다면**, 해당 토큰을 기준으로 **재귀적인** 토크나이즈를 수행합니다.
    - 먼저, 매치 토큰의 좌측 스팬을 **서브 스트링**으로 설정해 토크나이즈 함수에 입력합니다. 그리그 그 결과를 전체 결과 리스트에 추가합니다.
    - 이제, **매칭된 토큰**을 결과 값에 추가합니다.
    - **매치 토큰**이 한 문장 내에서 여러 개일 수 있으므로, 위 과정을 여러 번 반복합니다.
    - 이후 매치 토큰 우측 스팬을 **서브 스트링**으로 설정해 토크나이즈 함수에 입력합니다. 그리그 그 결과를 전체 결과 리스트에 추가합니다.

In [19]:
def tokenize(text: str, sorted_tokens: List[str], unknown_token="</u>"):
    """구축된 사전을 활용한 BPE 토크나이즈"""
    text = text.strip()

    if text == "":
        return list()
    if len(sorted_tokens) == 0:
        return [unknown_token]

    result = list()
    # 사전 내 등록 단어 순회
    for i in range(len(sorted_tokens)):
        token = re.escape(sorted_tokens[i])

        # 현재 순회 중인 단어가 입력 텍스트에 포함되는지 확인
        matched = [(m.start(0), m.end(0)) for m in re.finditer(token, text)]

        ## 단순히 포함되지 않은 것이라면, continue
        ## 토큰 리스트를 다 돌았음에도 포함되지 않은 것이라면, [unk] 반환
        if len(matched) == 0:
            if i == (len(sorted_tokens) - 1):
                return [unknown_token]
            else:
                continue

        ## 포함되면 해당 토큰의 시작점(들)을 저장
        ends = [m[0] for m in matched]
        print(f"[{text}] 매치 토큰: {token} / 인덱스: {ends}")

        start = 0
        for end in ends:
            # 매치 토큰 이전에 위치한 서브 스트링에 대한 토크나이즈 진행 및 결과 추가
            substring = text[start:end]
            print(f"[{text}] 서브 스트링: {substring} ({start}~{end})")
            result += tokenize(substring, sorted_tokens[i+1:])

            # 매치 토큰 추가
            result += [token]
            print(f"[{text}] 현재 토크나이즈 결과: {result}")

            # 매치 토큰 길이 만큼 start 인덱스 값 증가
            start = end + len(token)

        # 매치 토큰 이후에 위치한 서브 스트링에 대한 토크나이즈 진행 및 결과 추가
        remainder = text[start:]
        result += tokenize(remainder, sorted_tokens[i+1:])
        break
    return result

구현한 `tokenize` 함수를 활용해 새로운 입력 값에 대한 인코딩을 진행해보도록 합시다.

앞서 언급한대로 **입력 값**을 바꾸어 가며, **결과 값**을 찍어보는 것이 이해에 가장 도움이 됩니다.

In [20]:
tokens = get_tokens(vocab)

# 사전 내 토큰들 길이 순으로 정렬 후, 단어만 저장
sorted_tokens = sorted(tokens.items(), key=lambda x: (get_token_len(x[0]), x[1]), reverse=True)
sorted_tokens = [token for (token, _) in sorted_tokens]

print(f"사전 내 등록 단어:\n{sorted_tokens}\n")

words = ["한한국ㅑ한국한국국ㅖ", "사랑합니다</w>", "야식</w> 안먹고</w> 참아보기</w>"]

for word in words:
    print(f"입력 단어: {format(word)}\n")
    print(f"입력 단어 토큰화 결과: {tokenize(word, sorted_tokens)}\n")

사전 내 등록 단어:
['재미없다시간이</w>', '배우들이었는데</w>', '재미있었는데</w>', '재미없었는데</w>', '마지막장면이</w>', '재미없는영화</w>', '재미있는영화</w>', '재미없다는거</w>', '배우들이나와</w>', '지마지막장면</w>', '존재미있어요</w>', '습니다시간이</w>', '재미없다는게</w>', '니다니다니다</w>', '까지만들어서</w>', '재미없다는지</w>', '습니다가슴이</w>', '영화이야기는</w>', '드라마이네요</w>', '배우들이해도</w>', '드라마이라고</w>', '배우들이네요</w>', '배우들이지만</w>', '정말이영화는</w>', '영상영화라고</w>', '많이하는영화</w>', '영화이영화를</w>', '기대한다는거</w>', '가지만으로도</w>', '더라이영화로</w>', '파이영화라고</w>', '영화입니다</w>', '개인적으로</w>', '영화관에서</w>', '재미있어요</w>', '쓰레기영화</w>', '애니메이션</w>', '재밌었는데</w>', '재미있었다</w>', '모르겠지만</w>', '재미없어서</w>', '재미없었다</w>', '여주인공이</w>', '재미있는데</w>', '재미없다고</w>', '재미없네요</w>', '재미있어서</w>', '만들었는데</w>', '영화이지만</w>', '마지막장면</w>', '재미없는데</w>', '드라마지만</w>', '재미없다는</w>', '재미없는건</w>', '배우들에게</w>', '잼있었는데</w>', '재미있지만</w>', '재미있을듯</w>', '드라마이다</w>', '기대한다면</w>', '안되는영화</w>', '영화이네요</w>', '만들어지는</w>', '배우들이라</w>', '재미없는게</w>', '결말이지만</w>', '재미있는건</w>', '재미없는지</w>', '재미없다는거', '결말이라도</w>', '드라마이고</w>', '재미있는게</w>', '재미없는거</w>', '배우들이나</


입력 단어: 한한국ㅑ한국한국국ㅖ

[한한국ㅑ한국한국국ㅖ] 매치 토큰: 한국 / 인덱스: [1, 4, 6]
[한한국ㅑ한국한국국ㅖ] 서브 스트링: 한 (0~1)
[한] 매치 토큰: 한 / 인덱스: [0]
[한] 서브 스트링:  (0~0)
[한] 현재 토크나이즈 결과: ['한']
[한한국ㅑ한국한국국ㅖ] 현재 토크나이즈 결과: ['한', '한국']
[한한국ㅑ한국한국국ㅖ] 서브 스트링: ㅑ (3~4)
[한한국ㅑ한국한국국ㅖ] 현재 토크나이즈 결과: ['한', '한국', '</u>', '한국']
[한한국ㅑ한국한국국ㅖ] 서브 스트링:  (6~6)
[한한국ㅑ한국한국국ㅖ] 현재 토크나이즈 결과: ['한', '한국', '</u>', '한국', '한국']
[국ㅖ] 매치 토큰: 국 / 인덱스: [0]
[국ㅖ] 서브 스트링:  (0~0)
[국ㅖ] 현재 토크나이즈 결과: ['국']
입력 단어 토큰화 결과: ['한', '한국', '</u>', '한국', '한국', '국', '</u>']

입력 단어: 사랑합니다</w>

[사랑합니다</w>] 매치 토큰: 합니다</w> / 인덱스: [2]
[사랑합니다</w>] 서브 스트링: 사랑 (0~2)
[사랑] 매치 토큰: 사랑 / 인덱스: [0]
[사랑] 서브 스트링:  (0~0)
[사랑] 현재 토크나이즈 결과: ['사랑']
[사랑합니다</w>] 현재 토크나이즈 결과: ['사랑', '합니다</w>']
입력 단어 토큰화 결과: ['사랑', '합니다</w>']

입력 단어: 야식</w> 안먹고</w> 참아보기</w>

[야식</w> 안먹고</w> 참아보기</w>] 매치 토큰: 보기</w> / 인덱스: [17]
[야식</w> 안먹고</w> 참아보기</w>] 서브 스트링: 야식</w> 안먹고</w> 참아 (0~17)
[야식</w> 안먹고</w> 참아] 매치 토큰: 고</w> / 인덱스: [9]
[야식</w> 안먹고</w> 참아] 서브 스트링: 야식</w> 안먹 (0~9)
[야식</w> 안먹] 매치 토큰: 식</w> / 인덱스: [1]
[야식</w> 안먹] 

이상으로 **Byte-Pair Encoding**을 한국어에 적용했을 때, 사전이 어떤 식으로 구축되는지에 대해 알아보았습니다.

위 코드는 이해를 돕기 위해 작성된 것이기 때문에 실제 프로젝트에 **BPE**를 적용하고자 한다면, 저자의 [**공식 구현체**](https://github.com/rsennrich/subword-nmt)를 사용하시는 것이 좋을 것 같습니다.

### 참조
- [Lei Mao's Byte-Pair Encoding](https://leimao.github.io/blog/Byte-Pair-Encoding/)
- [Neural Machine Translation of Rare Words with Subword Units](https://arxiv.org/abs/1508.07909)
- [subword-nmt](https://github.com/rsennrich/subword-nmt)
- [nsmc](https://github.com/e9t/nsmc)